In [1]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
#splitedRdd.take(1)
splitedRdd = sc.parallelize(splitedRdd.take(2000))
splitedRdd.take(1)
#(uid,[[train],[test]]

[(18435,
  [[train(itemId=94874, context=context(ts=1420959850, city=2995, lat=42.897598, lon=-87.853897, moving=1, location=1, time_of_day=1)),
    train(itemId=14861, context=context(ts=1420982193, city=2995, lat=42.897644, lon=-87.853874, moving=1, location=2, time_of_day=2)),
    train(itemId=94874, context=context(ts=1420982271, city=2995, lat=42.897644, lon=-87.853874, moving=2, location=2, time_of_day=2)),
    train(itemId=94874, context=context(ts=1420982504, city=2995, lat=42.897675, lon=-87.853874, moving=2, location=2, time_of_day=2)),
    train(itemId=29345, context=context(ts=1420983397, city=2995, lat=42.897724, lon=-87.853844, moving=1, location=2, time_of_day=2)),
    train(itemId=54389, context=context(ts=1420983399, city=2995, lat=42.897686, lon=-87.853851, moving=3, location=2, time_of_day=2)),
    train(itemId=42869, context=context(ts=1420983404, city=2995, lat=42.897686, lon=-87.853851, moving=2, location=2, time_of_day=2)),
    train(itemId=42869, context=context

In [2]:
from operator import add
numRecList = 5
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList     
def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores

def evaluation_rr1(testItem, recList):
    scores = 0
    for i in range(len(recList)):
        if testItem == recList[i]:
            scores = 1.0 / (i+1)
            break
    return scores


def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0
                    num = num + 1  
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def evaluation_precision(testItem, recList):
    scores = 0
    if testItem in recList:
        scores = 1.0
    return scores  

def print_result(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation1.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()
    
def print_result2(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation5.txt','a')
    f.write('5000 users with duplicated evaluations percentage<30%')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()
def print_result3(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation6.txt','a')
    f.write('2000 users with duplicated evaluations train<300')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()
def print_result4(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation10.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()

In [11]:
def bayesian_context_location(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][5],t[1][1]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][5],t[1][1]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        trainList = [x for x in trainList if trainList[-1][2]-x[2] <= 43200]
        testList_items = [x[0] for x in testList if x[2]-testList[0][2] <= 43200]  #take only id for test set
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app_context = float(len(contextAppi))/len(appi)
            if p_app_context>0.3:
                p_app = topFiveSortedList(p_app,c[0],p*p_app_context*5)
            else:
                p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr1 = scores_rr1 / numTest
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRdd.map(bayesian_context_location)
print_result4(finalScore, 'bayesian_context_location_0.5day_time_window')

In [4]:
from collections import Counter
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest): 
        trainList = [x for x in trainList if trainList[-1][1][1]-x[1][1] <= 21600]
        testList_items = [x[0] for x in testList if x[1][1]-testList[0][1][1] <= 21600]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr1 = scores_rr1 / numTest
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = splitedRdd.map(mfuFunction)
print_result4(finalScore,'mfu_0.25day_time_window')

In [ ]:
from operator import add
from collections import Counter

def mfu_decay2(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        trainList = [x for x in trainList if trainList[-1][1][1]-x[1][1] <= 43200]
        testList_items = [x[0] for x in testList if x[1][1]-testList[0][1][1] <= 43200]   
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender =  [[t,0] for t in Recommender] 
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        decay = 1
        for i in range(len(trainList)):
            index = [x[0] for x in Recommender].index(trainList[i][0])
            Recommender[index][1] = Recommender[index][1] + decay
            decay = decay * 0.99 #0.9 is worse
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
    #return Recommender[:10],numTrain,testcount,numTest

finalScore = splitedRdd.map(mfu_decay2)
print_result4(finalScore,'mfu_decay2_0.5day_time_window')

In [ ]:
from operator import add
from collections import Counter

def mfu_mru_decay2(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        trainList = [x for x in trainList if trainList[-1][1][1]-x[1][1] <= 43200]
        testList_items = [x[0] for x in testList if x[1][1]-testList[0][1][1] <= 43200]  
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender =  [[t,0] for t in Recommender] 
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        RecommenderDuplicate1 =  [t[0] for t in trainList]        #take only id for train set
        Recommender1 = remove_duplicates(RecommenderDuplicate1)    #remove duplicate
        decay = 1
        for i in range(len(trainList)):
            index = [x[0] for x in Recommender].index(trainList[i][0])
            Recommender[index][1] = Recommender[index][1] + decay
            decay = decay * 0.99
        Recommender = [[t[0],t[1]] for t in Recommender if t[0] in Recommender1[:10]]
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
    #return Recommender[:10],numTrain,testcount,numTest

finalScore = splitedRdd.map(mfu_mru_decay2)
print_result4(finalScore,'mfu_mru_decay2')